# Data Visualization - Project 2
### Student names and IDs 
- Pedram Abdolahi Darestani - 202383919
- Javier Landin Cabrera - 202380154

## Importing libraries

In [26]:
import dash
import numpy as np
import pandas as pd 
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from datetime import datetime as dt
import dash_mantine_components as dmc
from mpl_chord_diagram import chord_diagram
from dash import html, dcc, callback, Output, Input
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas

## Loading and preprocessing data

In [27]:
data = pd.read_csv('data.csv')
data = data[data.columns.drop("Patient Id")]
time_columns = data.columns.drop(['Gender', 'Comorbidities', 'Age'])
data[data.columns.drop(['Gender', 'Comorbidities', 'Age'])] = data[data.columns.drop(['Gender', 'Comorbidities', 'Age'])].apply(pd.to_datetime)

ref = data['Emergency Dept Time']
# All the columns below have the timedelta64[ns] dtype and will be converted to hours later on
data['admission delay'] = data['Admission Time'] - data['Emergency Dept Time']
data['stay duration'] = data['Discharge Time'] - data['Admission Time']
data['ct delay'] = data['CT Scan Time'] - ref
data['tpa delay'] = data['TPA Time'] - ref
data['icu delay'] = data['ICU Arrival Time'] - ref
data['icu duration'] = data['ICU Checkout Time']- data['ICU Arrival Time']
data['nward delay'] = data['Neurology Ward Arrival Time'] - ref
data['neurologist delay'] = data['Neurologist Visit'] - ref
data['otherapist delay'] = data['Occupational Therapist Visit'] - ref
data['spathologist delay'] = data['Speech Pathologist Visit'] - ref
data['physio delay'] = data['Physiotherapist Visit'] - ref
data['dietitian delay'] = data['Dietitian Visit'] - ref
data['sworker delay'] = data['Social Worker Visit'] - ref
data['cardio delay'] = data['Cardiologist Visit'] - ref

data['year'] = data['Emergency Dept Time'].apply(lambda x : x.year)
data['month'] = data['Emergency Dept Time'].apply(lambda x : x.month)
data['week'] = data['Emergency Dept Time'].apply(lambda x : x.week)
data['day'] = data['Emergency Dept Time'].apply(lambda x : x.day)

# converting time deltas to hours (float)
tdelta_cols = data.select_dtypes(include='timedelta64[ns]').columns
data[tdelta_cols] = data[tdelta_cols]/pd.Timedelta(hours=1)

## App initialization

In [28]:
app = dash.Dash(
    __name__,
    suppress_callback_exceptions=True,
    meta_tags=[{"name": "DVP2", "content": "width=device-width, initial-scale=1"}],
)

## Visualization #1 - Time Patterns

In [29]:
delay_titles = {'admission delay': 'Admission Time',
 'ct delay': 'CT Scan Time',
 'tpa delay': 'TPA Time',
 'icu delay': 'ICU Time',
 'nward delay': 'Neurology Ward Time',
 'neurologist delay': 'Neurologist Visit Time',
 'otherapist delay': 'Occupational Therapist Visit Time',
 'spathologist delay': 'Speech Pathologist Visit Time',
 'physio delay': 'Physiotherapist Visit Time',
 'dietitian delay': 'Dietitian Visit Time',
 'sworker delay': 'Social Worker Visit Time',
 'cardio delay': 'Cardiologist Visit Time'}
duration_titles = {'stay duration':'Total Stay Duration', 'icu duration':'<b>ICU</b> Stay Duration'}
other_titles = {'Emergency Dept Time': 'Number of Check-ins to the <b>Emergency Department</b>',
 'Admission Time': 'Total Number of Admissions',
 'Discharge Time': 'Total Number of Discharges',
 'CT Scan Time': 'Total Number of <b>CT Scans</b>',
 'TPA Time': 'Total Number of <b>TPA</b>s',
 'ICU Arrival Time': 'Number of Check-ins to the <b>ICU</b>',
 'ICU Checkout Time': 'Number of Discharges from the <b>ICU</b>',
 'Neurology Ward Arrival Time': 'Number of Check-ins to the <b>Neurology Ward</b>',
 'Occupational Therapist Visit': 'Total Number of <b>Occupational Therapist</b> Visits',
 'Speech Pathologist Visit': 'Total Number of <b>Speech Pathologist</b> Visits',
 'Physiotherapist Visit': 'Total Number of <b>Physiotherapist</b> Visits',
 'Dietitian Visit': 'Total Number of <b>Dietitian</b> Visits',
 'Social Worker Visit': 'Total Number of <b>Social Worker</b> Visits',
 'Cardiologist Visit': 'Total Number of <b>Cardiologist</b> Visits',
 'Neurologist Visit': 'Total Number of <b>Neurologist</b> Visits'}
month_tick_dic = dict(tickmode = 'array', tickvals = np.arange(12),ticktext = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'])

default_time_range = [pd.to_datetime('1/3/2022 18:02'), pd.to_datetime('1/1/2024 8:22')]
@app.callback(
    Output("viz1_time_pattern", "figure"),
    [
        Input("date-picker-select", "value"),
        Input("clinic-select", "value"),
        Input("scale-picker-select", "value"),
        Input("initial-age-select", "value"),
        Input("end-age-select", "value"),
        Input("gender-picker-select", "value"),
        Input("comor-picker-select", "value")
    ],
)
def plot_viz1(time_range=default_time_range, event='admission delay', 
              scale='year', age1=27, age2=98, gender=None, comorbidity=None):
    """
    Plots histogram of the occurrence of an event in a time interval
    :param time1: Beginning of the time interval - must have a DateTime format
    :param time2: End of the time interval - must have a DateTime format
    :param event: Histogram of which event/admission is to be plotted
    :param scale: Scale of the histogram = 'year' or 'month'
    :param age1: Beginning of the age interval 
    :param age2: End of the age interval 
    :param gender: 'M', 'F', and None for both
    :param comorbidity: True, False, or None for both
    :return: 
    """
    time1 = pd.to_datetime(time_range[0])
    time2 = pd.to_datetime(time_range[1])
    year_tick_dic = dict(tickmode = 'array', tickvals = np.arange(data['year'].nunique()),ticktext = data['year'].unique())
    x = data
    # filtering gender
    if gender not in [None, 'All']:
        x = x[x['Gender']==gender]
    # filtering comorbidity
    if comorbidity not in [None, 'All']:
        if comorbidity == 'Yes':
            x = x[x['Comorbidities']==True]
        else:
            x = x[x['Comorbidities']==False]
    # filtering event, time, and age range
    '''
    '''
    if event in delay_titles.values():
        for key, value in delay_titles.items():
            if value == event:
                event = key
                break
    
    if type(age1) != int: age1=0
    if type(age2) != int: age2=0
    
    if event in delay_titles.keys():
        x = x[~x[event].isna()][(x['Emergency Dept Time']>=time1) & (x['Emergency Dept Time']<=time2) & (x['Age']>=age1) & (x['Age']<=age2)]
        x = x.groupby(scale).mean(numeric_only=True).reset_index()[event]
        title = f'Mean Time Difference Between Arrival to the <br> <b>Emergency Department</b> and <b>{delay_titles[event]}</b> for Every <b>{scale.title()}</b>'
        yaxis_title = "Mean Time Difference (Hours)"
        xaxis_title = "Month" if scale=='month' else 'Year'
    elif event in duration_titles.keys():
        x = x[~x[event].isna()][(x['Emergency Dept Time']>=time1) & (x['Emergency Dept Time']<=time2) & (x['Age']>=age1) & (x['Age']<=age2)]
        x = x.groupby(scale).mean(numeric_only=True).reset_index()[event]
        title = f'Mean <b>{duration_titles[event]}</b> for Every <b>{scale.title()}</b>'
        yaxis_title = "Mean Duration (Hours)"
        xaxis_title = "Month" if scale=='month' else 'Year'
    else:
        x = x[~x[event].isna()][(x[event]>=time1) & (x[event]<=time2) & (x['Age']>=age1) & (x['Age']<=age2)]
        x = x.groupby(scale).count().reset_index()[event]
        title = f'{other_titles[event]} for Every <b>{scale.title()}</b>'
        yaxis_title = "Count"
        xaxis_title = "Month" if scale=='month' else 'Year'
        
    fig = go.Figure()    
    fig.add_trace(go.Bar(x=np.arange(12) if scale=='month' else np.arange(data['year'].nunique()), y=x))
    fig.update_layout(xaxis_range=[-0.5, 11.5] if scale=='month' else [-0.5, len(data['year'].unique())-0.5],
                      title={'text': title, 'x':0.5}, xaxis_title=xaxis_title, yaxis_title=yaxis_title,
                      xaxis = month_tick_dic if scale=='month' else year_tick_dic
                      )
    return fig
plot_viz1(event='ct delay', scale='year', age1=50, age2=55)

## Visualization #2 - Delay Correlations

In [30]:
proper_colnames = {'admission delay': 'Admission Delay',
 'stay duration': 'Stay Duration',
 'ct delay': 'CT Scan Delay',
 'tpa delay': 'TPA Delay',
 'icu delay': 'ICU Delay',
 'icu duration':'ICU Duration',
 'nward delay': 'Neurology Ward Arrival Delay',
 'neurologist delay': 'Neurologist Visit Delay',
 'otherapist delay': 'Occupational Therapist Visit Delay',
 'spathologist delay': 'Speech Pathologist Visit Delay',
 'physio delay': 'Physiotherapist Visit Delay',
 'dietitian delay': 'Dietitian Visit Delay',
 'sworker delay': 'Social Worker Visit Delay',
 'cardio delay': 'Cardiologist Visit Delay', 
 'year':'Year',
 'month':"Month",
 'week':"Week",
 'day':"Day"}
@app.callback(
    Output("viz2_corr_mat", "figure"),
    [
        Input("date-picker-select", "value"),
        Input("initial-age-select", "value"),
        Input("end-age-select", "value"),
        Input("gender-picker-select", "value"),
        Input("comor-picker-select", "value")
    ],
)
def plot_viz2(time_range=default_time_range,
              age1=27, age2=98, gender=None, comorbidity=None):
    time1 = pd.to_datetime(time_range[0])
    time2 = pd.to_datetime(time_range[1])
    x = data
    # filtering time
    x = x[(x['Emergency Dept Time']>=time1) & (x['Emergency Dept Time']<=time2)]
    # filtering age
    if type(age1) != int: age1=0
    if type(age2) != int: age2=0
    if age1 > age2:
        age1, age2 = age2, age1
    x = x[(x['Age']>=age1) & (x['Age']<=age2)]

    # filtering gender
    if gender not in [None, 'All']:
        x = x[x['Gender']==gender]
    # filtering comorbidity
    if comorbidity not in [None, 'All']:
        if comorbidity == 'Yes':
            x = x[x['Comorbidities']==True]
        else:
            x = x[x['Comorbidities']==False]
    
    x = x.rename(proper_colnames, axis=1)
    x = x[x.columns.drop(['Day', 'Week', 'Year', 'Month', 'Age', 'Comorbidities', ])]
    cmat = x.corr(numeric_only=True)
    cmat = cmat.round(decimals=2)
    fig = px.imshow(cmat, text_auto=True, width=800, height=800)
    fig.update_layout(
        xaxis_side='bottom',
        coloraxis_colorbar={
            'ypad': 0,
            'title': 'Correlation Value',
            'title_side': 'right',
            'len': 0.5,
            'thickness': 8,
            'x': 1,
            'y': 0.5
        },
        xaxis={'tickangle': -45},
        margin=dict(l=0, r=0, t=0, b=0)
    )
    # The colorbar is adding an ugly padding on top of the matrix. I haven't been able to reduce it
    fig.update(layout_coloraxis_showscale=False)
    return fig

plot_viz2()

## Visualization #3 - Scatterplot

In [31]:
@app.callback(
    Output("viz3_scatter", "figure"),
    [
        Input("x-axis-select", "value"),
        Input("y-axis-select", "value"),
        Input("date-picker-select", "value"),
        Input("initial-age-select", "value"),
        Input("end-age-select", "value"),
        Input("gender-switch", "checked"),
        Input("comor-switch", "checked")
    ],
)
def plot_viz3(xaxis, yaxis, time_range=default_time_range, age1=27, age2=98, gender=False, comorbidity=False):
    # plot a scatter of feature x against feature y - filter time and age range - gender defines color - comorbidity defines shape
    title = f'Patient <b>{xaxis}</b> against <b>{yaxis}</b>'
    xaxis_label = xaxis + ' (Hours)'
    yaxis_label = yaxis + ' (Hours)'
    
    if xaxis in delay_titles.values():
        for key, value in delay_titles.items():
            if value == xaxis:
                xaxis = key
                break
    if yaxis in delay_titles.values():
        for key, value in delay_titles.items():
            if value == yaxis:
                yaxis = key
                break
    
    time1 = pd.to_datetime(time_range[0])
    time2 = pd.to_datetime(time_range[1])
    x = data
    x = x[(x['Emergency Dept Time']>=time1) & (x['Emergency Dept Time']<=time2)]
    x = x[(x['Age']>=age1) & (x['Age']<=age2)]
    if gender==False and comorbidity==False:
        fig = px.scatter(x, x=xaxis, y=yaxis)
    elif gender==False and comorbidity==True:
        fig = px.scatter(x, x=xaxis, y=yaxis, symbol='Comorbidities', symbol_sequence=['circle', 'triangle-up-dot'])
    elif gender==True and comorbidity==False:
        fig = px.scatter(x, x=xaxis, y=yaxis, color='Gender')
    else:
        fig = px.scatter(x, x=xaxis, y=yaxis, symbol='Comorbidities', symbol_sequence=['circle', 'triangle-up-dot'], color='Gender')

    # Update layout with axis labels
    fig.update_layout(
        title={'text': title, 'x':0.5},
        xaxis_title=xaxis_label,
        yaxis_title=yaxis_label
    )
    
    return fig


# plot_viz3()
plot_viz3('spathologist delay', 'physio delay')

## Visualization #4 - Chord Diagram

In [32]:
def plot_viz4(data, time_columns):
    times = data[time_columns]
    time_to_col = dict(enumerate(time_columns))
    col_to_time = {time_to_col[b]:b for b in time_to_col.keys()}
    links = pd.DataFrame(np.zeros((len(time_columns), len(time_columns))))

    sorted_indices = times.apply(lambda x: x.sort_values().dropna().index.tolist(), axis=1)

    for sorted_index_list in sorted_indices:
        for j in range(len(sorted_index_list) - 1):
            source = col_to_time[sorted_index_list[j]]
            target = col_to_time[sorted_index_list[j + 1]]
            links.iloc[source, target] += 1

    links = links.astype('Int32')
    llist = [{'source': i, 'target': j, 'value': links.iloc[i, j]}
            for i in range(len(links)) for j in range(len(links))]

    sandat = pd.DataFrame(llist)
    sandat = sandat[sandat['value'] > 75]
    sandat = sandat.sort_values('source', ascending=True)
    labels = time_columns
    if links.shape[0] == 15:
        links = links[links.columns.drop(13)]
        links = links.drop(13)
    names = ['Emergency Entry', 'Admission', 'Discharge',
        'CT Scan', 'TPA', 'ICU Arrival', 'ICU Checkout',
        'Neurology Ward Arrival', 'Occupational Therapist Visit',
        'Speech Pathologist Visit', 'Physiotherapist Visit', 'Dietitian Visit',
        'Social Worker Visit', 'Neurologist Visit']
    for i in range(links.shape[0]):
        for j in range(links.shape[1]):
            if links.iloc[i,j] <20:
                links.iloc[i,j] = 0

    fig, ax = plt.subplots(figsize=(10, 10))
    order = [3, 0, 1, 7, 6, 5, 4, 8, 2, 9, 10, 11, 12, 13]# 13 is the previous 14. the 13th column was the cardio that was removed.
    chord_diagram(links.astype(int).to_numpy(), names=names, directed=True, ax=ax, cmap='Set1', order=order); 

    canvas = FigureCanvas(fig)
    canvas.draw()
    buf = canvas.buffer_rgba()
    fig_array = np.asarray(buf)

    plt.close(fig)
    plotly_fig = px.imshow(fig_array)
    plotly_fig.update_layout(width=600, height=600, margin=dict(l=10, r=10, b=10, t=10))
    plotly_fig.update_xaxes(showticklabels=False).update_yaxes(showticklabels=False)
    return plotly_fig

plot_viz4(data, time_columns)

c:\Users\carli\anaconda3\Lib\site-packages\mpl_chord_diagram\utilities.py:167: RuntimeWarning:

invalid value encountered in divide



# Dash modules
## Description Card
A card that shows initial information

In [33]:
def description_card():
    """

    :return: A Div containing dashboard title & descriptions.
    """
    return html.Div(
        id="description-card",
        children=[
            dmc.Title(f"    Hospital Analytics", order=2),
            html.Div(
                dmc.Text("Explore mean time intervals across departments, view correlation, compare and monitor patient flow in the hospital system.", size="sm"),
            ),
        ],
    )

## Controls Section

In [34]:
event_list = ['Emergency Department Arrival', 'Admission', 'Discharge', 'CTScan', 'TPA', 'ICU Checkout', 'Neurology Ward Arrival','Occupational Therapist Visit', 'Speech Pathologist Visit', 'Dietitian Visit', 'Social Worker Visit', 'Cardiologist Visit', 'Neurologist Visit']
scale_list = [["year", "Year"], ["month", "Month"]]
gender_list = [["M", "M"], ["F", "F"], ["All", "All"]]
comor_list = [["Yes", "Comorbidities"], ["No", "No Comorbidities"], ["All", "All"]]
def generate_controls_viz1():
    """
    :return: A Div containing controls for graphs.
    """
    return html.Div(
        id="control-card",
        children=[
            html.Br(),
            dmc.Select(
                label="Select Department",
                id="clinic-select",
                data=[{"label": i, "value": i} for i in list(delay_titles.values())],
                value=list(delay_titles.values())[0], # TODO: fix input values
            ),
            html.Br(),
            dmc.DateRangePicker(
                label="Select Check-In Time Interval",
                id="date-picker-select",
                maxDate=dt(2024, 1, 1),
                minDate=dt(2022, 1, 3),
                value=[dt(2022, 1, 3), dt(2024, 1, 1)]
            ),
            html.Br(),
            dmc.RadioGroup(
                [dmc.Radio(l, value=k) for k, l in scale_list], 
                label="Date Scale",
                value='year', 
                id="scale-picker-select"
            ),
            html.Br(),
            dmc.NumberInput(
                label='Minimum Age',
                id="initial-age-select",
                value=0
            ),
            html.Br(),
            dmc.NumberInput(
                label='Maximum Age',
                id="end-age-select",
                value=100
            ),
            html.Br(),
            dmc.RadioGroup(
                [dmc.Radio(l, value=k) for k, l in gender_list], 
                value="All", 
                id="gender-picker-select"
            ),
            html.Br(),
            dmc.RadioGroup(
                [dmc.Radio(l, value=k) for k, l in comor_list], 
                value="All", 
                id="comor-picker-select"
            ),
            html.Br(),
            html.Br(),
            html.Br(),
            # html.Div(
            #     id="reset-btn-outer",
            #     children=html.Button(id="reset-btn", children="Reset", n_clicks=0),
            # ),
        ],
    )

def generate_controls_viz2():
    """
    :return: A Div containing controls for graphs.
    """
    return html.Div(
        id="control-card",
        children=[
            html.Br(),
            dmc.DateRangePicker(
                label="Select Check-In Time Interval",
                id="date-picker-select",
                maxDate=dt(2024, 1, 1),
                minDate=dt(2022, 1, 3),
                value=[dt(2022, 1, 3), dt(2024, 1, 1)]
            ),
            html.Br(),
            dmc.NumberInput(
                label='Minimum Age',
                id="initial-age-select",
                value=0
            ),
            html.Br(),
            dmc.NumberInput(
                label='Maximum Age',
                id="end-age-select",
                value=100
            ),
            html.Br(),
            dmc.RadioGroup(
                [dmc.Radio(l, value=k) for k, l in gender_list], 
                value="All", 
                id="gender-picker-select"
            ),
            html.Br(),
            dmc.RadioGroup(
                [dmc.Radio(l, value=k) for k, l in comor_list], 
                value="All", 
                id="comor-picker-select"
            ),
            html.Br(),
            html.Br(),
            html.Br(),
            # html.Div(
            #     id="reset-btn-outer",
            #     children=html.Button(id="reset-btn", children="Reset", n_clicks=0),
            # ),
        ],
    )
def generate_controls_viz3():
    """
    :return: A Div containing controls for graphs.
    """
    return html.Div(
        id="control-card",
        children=[
            html.Br(),
            dmc.Select(
                label="X Axis",
                id="x-axis-select",
                data=[{"label": i, "value": i} for i in list(delay_titles.values())],
                value=list(delay_titles.values())[0], # TODO: fix input values
            ),
            dmc.Select(
                label="Y Axis",
                id="y-axis-select",
                data=[{"label": i, "value": i} for i in list(delay_titles.values())],
                value=list(delay_titles.values())[0], # TODO: fix input values
            ),
            html.Br(),
            dmc.DateRangePicker(
                label="Select Check-In Time Interval",
                id="date-picker-select",
                maxDate=dt(2024, 1, 1),
                minDate=dt(2022, 1, 3),
                value=[dt(2022, 1, 3), dt(2024, 1, 1)]
            ),
            html.Br(),
            dmc.NumberInput(
                label='Minimum Age',
                id="initial-age-select",
                value=0
            ),
            html.Br(),
            dmc.NumberInput(
                label='Maximum Age',
                id="end-age-select",
                value=100
            ),
            html.Br(),
            dmc.Switch(
                label="Gender", 
                id="gender-switch",
                checked=False
            ),
            html.Br(),
            dmc.Switch(
                label="Comorbidities", 
                id="comor-switch",
                checked=False
            ),
            html.Br(),
            html.Br(),
            html.Br(),
            # html.Div(
            #     id="reset-btn-outer",
            #     children=html.Button(id="reset-btn", children="Reset", n_clicks=0),
            # ),
        ],
    )
def generate_controls_viz4():
     return html.Div(
        id="control-card",
        children=[
        html.Br(id='empty')]
    )

## App Layout

In [35]:
viz_titles = {"viz1":"viz1_time_pattern", "viz2":"viz2_corr_mat", "viz3":"viz3_scatter", "viz4":"viz4_chord"}
app.title = "Hospital Analytics Dashboard"

server = app.server
app.layout = html.Div(
    id="app-container",
    children=[
        # Banner with Mantine Segmented Control
        html.Div(
            id="banner",
            className="banner",
            children=[]
        ),
        # Two-column layout using Grid and Col
        dmc.Grid(
            children=[
                dmc.Col(
                    span=3,  # Set span to 4 for left column (out of 12)
                    id="control-gen",
                    children=[
                        description_card(),
                        generate_controls_viz1()
                    ],
                ),
                dmc.Col(
                    span=9,  # Set span to 8 for right column (out of 12)
                    children=[
                                dmc.SegmentedControl(
                                    id="viz-control",
                                    value="viz1",
                                    data=[
                                        {"value": "viz1", "label": "Time Patterns"},
                                        {"value": "viz2", "label": "Time Correlation"},
                                        {"value": "viz3", "label": "Time Scatter Plot"},
                                        {"value": "viz4", "label": "Chord Diagram"}],
                                    size="md",
                                    fullWidth=True
                                ),
                                html.Div(
                                    id="viz-div",
                                    children=[
                                        dcc.Graph(id="viz1_time_pattern")
                                    ],
                                ),
                            ],
                ),
            ]
        ),
    ],
)
@callback([Output("viz-div", "children"), Output("control-gen", "children")], Input("viz-control", "value"))
def update_graph_and_controls(viz_value):
    if viz_value:
        graph_id = viz_titles.get(viz_value)
        if graph_id:
            if viz_value == 'viz4':
                graph = [html.Hr(),
                         html.Br(),
                         html.Br(),
                         html.Br(),
                         html.Br(),  
                         dcc.Loading(html.Div(id='placeholder-viz4'), type='circle')]
            else:
                graph = [html.Hr(), dcc.Graph(id=graph_id)]
        else:
            graph = []
    else:
        graph = []

    if viz_value == "viz1":
        controls = [description_card(),generate_controls_viz1()]
    elif viz_value == "viz2":
        controls = [description_card(),generate_controls_viz2()]
    elif viz_value == "viz3":
        controls = [description_card(),generate_controls_viz3()]
    elif viz_value == "viz4":
        controls = [description_card(),generate_controls_viz4()]
    else:
        controls = []

    return graph, controls

@callback(Output('placeholder-viz4', 'children'), [Input('viz-control', 'value')])
def render_viz4(viz_value):
    if viz_value == 'viz4':
        return dcc.Graph(figure=plot_viz4(data, time_columns))
    return []

# Run app

In [36]:
app.run_server(debug=True, jupyter_mode="external")

Dash app running on http://127.0.0.1:8050/
